In [1]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# get links in 9 way dataset
with open('9_way_list-Copy1.txt', 'r') as file:
    composers_9 = file.readlines()

composers_9 = ['https://imslp.org/wiki/Category:' + composer[:-1] for composer in composers_9]
print(composers_9)

In [ ]:
# get links in 100 way dataset
with open('100_way_list-Copy1.txt', 'r') as file:
    composers_100 = file.readlines()

composers_100 = ['https://imslp.org/wiki/Category:' + composer[:-1] for composer in composers_100]
print(composers_100)

In [ ]:
# code the previous team uses to find all the urls for a composer

def find_score_url(url):
    """
    This function takes in one arguments:
    1) url, a composer url page to parse
    This function returns a list of piece url for a specific composer.
    """
    list_page = [url]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # add everything on the first page to the list
    result = soup.find_all('div', class_="jq-ui-tabs" )
    if result == []:
        return "badlink"
    scores = result[0].find_all('a',class_='categorypagelink')
    url_list = [link.get('href') for link in scores]
    # check sub-page
    list_page = find_next_score_url(url)
    if list_page != []:
        for page in list_page:
            r = requests.get(page)
            soup = BeautifulSoup(r.text, "html.parser")
            url_list += [link.get('href') for link in soup.find_all('a',class_='categorypagelink')]
    return url_list

def find_next_score_url(url):
    """
    Helper function for find_score_url
    """
    result = []
    while True:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        url_list = [link.get('href') for link in soup.find_all('a',class_='categorypaginglink',string='next 200')]
        if url_list == []:
            break
        url = 'http://imslp.org' + url_list[-1]
        result.append(url)
    return result

In [ ]:
urls_9 = []
for composer in composers_9:
    urls_9+= find_score_url(composer)

urls_9 = ['https://imslp.org' + s for s in urls_9]

In [ ]:
urls_100 = []
for composer in composers_100:
    urls_100 += find_score_url(composer)

urls_100 = ['https://imslp.org' + s for s in urls_100]

In [ ]:
def save_list_to_file(lst, filename):
    with open(filename, 'w') as f:
        for item in lst:
            f.write(str(item) + '\n')
        
# create file for all urls from 9-way composers
save_list_to_file(urls_9, '9_way_urls_for_all_scores.txt')

# create file for all urls from 100-way composers
save_list_to_file(urls_100, '100_way_urls_for_all_scores.txt')